In [1]:

from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import numpy as np
import zipfile



import os
import shutil

In [2]:
# Open url for scraping. All data sources are stored within a table. All target data files have the
# same naming scheme, we will take advantage of this

In [3]:

url = 'http://www.fueleconomy.gov/feg/download.shtml'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')


reg = 'epadata/\d{2}data.zip' ##luckily our target has a unique identifier for href
suf = 'http://www.fueleconomy.gov/feg/'

In [4]:
# Find target data sources within html page

In [5]:
links = soup.find_all('a')
zlist = []
for tag in links:
    link = tag.get('href',None)
    if link is not None:
        if re.match(reg, link):
            zlist.append(link)

In [6]:
# Download data to current working directory. These files are zipped. Unzip files.

In [7]:
filelst = []
for x in zlist:
    url = suf+x
    localfn = url.split('/')[-1]
    yearno = int(localfn[:2])
    if yearno < 50 and yearno > 00:
        r = requests.get(url, stream=True)
        with open(localfn, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
        zip_ref = zipfile.ZipFile(localfn, 'r')
        lcl = "dldir"+localfn
        filelst.append(lcl)
        zip_ref.extractall(lcl)
        zip_ref.close()
       
        


In [8]:
# create directory to store data

In [9]:
datdir = "shibe_data"
if not os.path.exists(datdir):
    os.makedirs(datdir)

In [10]:
# extract downloaded data to single directory. Rename files so that they have uniform names

In [11]:
reg = "(.xlsx|.xls|.csv)$"
for item in filelst:
    for filename in os.listdir(item):
        curname =  item + "\\" + filename 
        
        
        m = re.search(reg, filename)
        if m:
            newname = datdir + "\\" + item[:-4] + m.group(0)
            os.rename(curname, newname)


In [12]:
# clean up files and directories created while scraping data

In [13]:

lst = os.listdir(os.getcwd())
for x in lst:
    m = re.search(".zip$", x)
    if m:
        n = re.search("^dldir",x)
        if n:
            shutil.rmtree(x)
        else:
            os.remove(x)


In [19]:
# Reads data files into Pandas Dataframes according to file type (.csv .xlx .xlsx). The df are stored in a temporary dict 
# according to their respective years. The original zip file from the EPA site does not give year in YYYY format, so we have 
# chosen not to use the YYYY format until this point where it becomes necessary and after we have gleaned out only the years 
# prefixed by "20-"
# Some lines of the original data files are formed incorrectly. Adding the option "error_bad_lines=False" skips these lines

In [198]:
%%capture
datfiles = os.listdir(datdir)
pdfiles = {}
for x in datfiles:
    y = re.search("[0-9]{2}",x)
    year = "20" + y.group(0)
    cs = re.search(".csv$", x)
    if cs:
        newp = pd.read_csv(datdir + "\\" + x, error_bad_lines=False)
    else:
        newp = pd.read_excel(datdir + "\\" + x)
        
    pdfiles[year] = newp
    

In [121]:
# Went through all the data files from 2001-2018 and found all the persisting variables and listed their aliases in a dict, vars

In [194]:
vars['Class'] = ['CLASS','Carline Class Desc', 'Class']
vars['Manufacturer'] = ['MFR', 'Manufacturer', 'Mfr Name', 'Mfr Name ']
vars['carline name'] = ['CAR LINE', 'Carline', 'carline name']
vars['cmb'] = ['COMB MPG (GUIDE)', 'Comb FE (Guide) - Conventional Fuel', 'cmb']
vars['cty'] = ['CITY MPG (GUIDE)', 'City FE (Guide) - Conventional Fuel', 'cty']
vars['cyl'] = ['# Cyl', 'NUMB CYL', 'cyl']
vars['displ'] = ['DISPLACEMENT', 'Eng Displ', 'displ']
vars['fcost'] = ['ANL FL CST', 'Annual Fuel1 Cost - Conventional Fuel', 'fcost']
vars['fl'] = ['FUEL TYPE', 'Fuel Usage Desc - Conventional Fuel', 'fl']
vars['hwy'] = ['HWY MPG (GUIDE)', 'Hwy FE (Guide) - Conventional Fuel', 'hwy']
vars['trans'] = ['TRANS', 'Trans as listed in FE Guide (derived from col AA thru AF)', 'Trans in FE Guide (MFR entered for data entered after May 13 2011)', 'Transmission', 'trans']
vars['ucmb'] = ['Comb Unadj FE - Conventional Fuel', 'UNRND COMP (EPA)', 'ucmb']
vars['ucty'] = ['City Unadj FE - Conventional Fuel', 'UNRND CITY (EPA)', 'ucty']
vars['uhwy'] = ['Hwy Unadj FE - Conventional Fuel', 'UNRND HWY (EPA)', 'uhwy']
vars['drv'] = ['DRIVE SYS', 'Drive Sys', 'drv']

['Carline Class Desc', 'Class']

In [200]:
varlst = list(vars.keys())
yearlst = list(pdfiles.keys())


for z in yearlst:
    curpd = pdfiles[z]
    cols = list(curpd.columns.values)
    checklst = []
    rename = []
    
    for x in cols:
        for y in varlst:
            if x in vars[y]:
                checklst.append(x)
                rename.append(y)
    curpd = curpd[checklst]
    curpd.columns = rename
    pdfiles[z] = curpd[varlst]


In [208]:
pdfiles["2017"]


,Class,Manufacturer,carline name,displ,cyl,trans,cty,hwy,cmb,ucty,uhwy,ucmb,fl,fcost,drv
0,Two Seaters,Honda,NSX,3.5,6,Auto(AM-S9),21,22,21,28.7000,30.2000,29.3561,Gasoline (Premium Unleaded Required),2000,A
1,Two Seaters,FCA US LLC,4C,1.8,4,Auto(AM6),24,34,28,28.7000,45.7000,34.4702,Gasoline (Premium Unleaded Recommended),1500,R
2,Two Seaters,aston martin,V12 Vantage S,6.0,12,Auto(AM7),12,18,14,14.7893,25.3011,18.1901,Gasoline (Premium Unleaded Recommended),3000,R
3,Two Seaters,aston martin,V12 Vantage S,6.0,12,Manual(M7),10,16,12,12.2894,22.3680,15.4150,Gasoline (Premium Unleaded Recommended),3500,R
4,Two Seaters,Volkswagen Group of,R8,5.2,10,Auto(AM-S7),14,22,17,17.2263,29.0421,21.0869,Gasoline (Premium Unleaded Recommended),2450,A
5,Two Seaters,Volkswagen Group of,R8 Spyder,5.2,10,Auto(AM-S7),14,22,17,17.2263,29.0421,21.0869,Gasoline (Premium Unleaded Recommended),2450,A
6,Two Seaters,Volkswagen Group of,TT Roadster quattro,2.0,4,Auto(AM-S6),23,30,26,28.7495,41.8440,33.4616,Gasoline (Regular Unleaded Recommended),1400,A
7,Two Seaters,General Motors,CORVETTE,6.2,8,Auto(S8),13,23,16,16.4000,29.5000,20.4957,Gasoline (Premium Unleaded Required),2600,R
8,Two Seaters,General Motors,CORVETTE,6.2,8,Auto(S8),15,26,19,18.7883,36.4554,24.0284,Gasoline (Premium Unleaded Required),2200,R
9,Two Seaters,General Motors,CORVETTE,6.2,8,Manual(M7),15,22,18,18.5952,30.7737,22.6242,Gasoline (Premium Unleaded Required),2350,R
